# Imports and settings

In [1]:
%matplotlib inline
%env JAX_ENABLE_X64=1
%env JAX_PLATFORM_NAME=cpu

env: JAX_ENABLE_X64=1
env: JAX_PLATFORM_NAME=cpu


In [2]:
import jax
import jax.numpy as jnp
from jax import jit, grad
from jax.example_libraries import optimizers

In [3]:
import os
import pickle
import time
import itertools

from collections import defaultdict
from functools import reduce, partial

In [4]:
from typing import Any, List, DefaultDict,\
    Dict, Tuple, Callable, Optional

## Types

In [5]:
# type alias
PRNGKeyArray = Any
DeviceArray = jnp.DeviceArray

# Random Data Generation

## MPS

In [6]:
def random_mps(
    key: PRNGKeyArray,
    size: int,
    local_dim: int,
    bond_dim: int,
    dtype: Optional[jnp.double]
    ) -> List[DeviceArray]:
    """
    Generate a random MPS where each core tensor
    is drawn i.i.d. from a uniform distribution 
    between -1 and 1.

    Input:
    ------
    key:        The random key.
    size:       The size (length) of an MPS.
    local_dim:  The local dimension size.
    bond_dim:   The bond dimension size.
    dtype:      The type of data to return.
    """
    # initialize MPS data collection
    mps = []
     
    for i in range(size):
        key, _ = jax.random.split(key)
        if i == 0:  # left most tensor
            tensor = jax.random.uniform(
                key, shape=(1, local_dim, bond_dim), minval=-1, maxval=1, dtype=dtype)
        elif i == size-1:  # right most tensor
            tensor = jax.random.uniform(
                key, shape=(bond_dim, local_dim, 1), minval=-1, maxval=1, dtype=dtype)
        else:  # middle tensors
            tensor = jax.random.uniform(
                key, shape=(bond_dim, local_dim, bond_dim), minval=-1, maxval=1, dtype=dtype)
        mps.append(tensor)

    return mps

In [7]:
def dot(mps1: List[DeviceArray], mps2: List[DeviceArray]) -> jnp.double:
    """
    Dot product of an MPS with another mps.
    --A1----A2--...--An-- (MPS1)
      |     |        |
    
      |     |        |
    --B1----B2--...--Bn-- (MPS2)
    """
    # contracts individual components
    cdot = lambda x, y: jnp.einsum('pqr,uqv->purv', x, y)
    # multiply two neighbouring tensors
    mult = lambda x, y: jnp.einsum('purv,rvts->puts', x, y)
    # contract all
    res = reduce(mult, jax.tree_multimap(cdot, mps1, mps2))
    return res.squeeze()

def mps_norm(mps: List[DeviceArray]) -> jnp.double:
    """Computing the squared norm of an MPS"""
    mps_conj = jax.tree_map(jnp.conj, mps)
    return dot(mps, mps_conj)

## Samples

In [8]:
def _random_sample(
    key: PRNGKeyArray, 
    num_factors: int, 
    local_dim: int, 
    dtype=jnp.double) -> DeviceArray:
    """Generate a single sample with a number of factors
    where each factor is generated from a Normal distribution.
    """
    keys = jax.random.split(key, num=num_factors)
    func = lambda k: jax.random.normal(k, (local_dim,), dtype)
    return jax.vmap(func)(keys)

def random_samples(
    key: PRNGKeyArray, 
    sample_size: int, 
    num_factors: int, 
    local_dim: int, 
    dtype=jnp.double) -> DeviceArray:
    """Genarate random samples of a specific size"""
    keys = jax.random.split(key, num=sample_size)
    return jax.vmap(lambda k: _random_sample(k, num_factors, local_dim, dtype))(keys)

In [9]:
def sample_as_mps(sample: DeviceArray) -> List[DeviceArray]:
    """
    Represent a data sample as an MPS. Useful for contracting with another MPS.
    |    |    |       |        |     |     |          |
    x1   x2   x3 ...  xn  => --x1----x2----x3-- ... --xn--
    """
    return list(sample[:,jnp.newaxis,:,jnp.newaxis])

def dot_samples(mps: List[DeviceArray], samples: DeviceArray) -> DeviceArray:
    """Apply dot product to many samples"""
    return jax.vmap(lambda s: dot(mps,sample_as_mps(s)))(samples)

# Helpers

In [10]:
def save_pkl(file_path: str, data: Any) -> None:
    dir_name = os.path.dirname(file_path)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

In [11]:
def idict() -> DefaultDict:
    """Infinitely nested dict"""
    return defaultdict(idict)

def idict2dict(dic) -> Dict:
    if isinstance(dic, defaultdict):
        dic = {k: idict2dict(v) for k, v in dic.items()}
    return dic

In [12]:
def make_dirs(root_dir: str, noise_level: float) -> None:
    # A timestamp used in the experiments to store data
    time_stamp = time.strftime('%Y%m%d', time.localtime())
    exp_dir = os.path.join(f'{root_dir}/{time_stamp}-noise-{noise_level}-{WEIGHT_INIT}')
    lrn_dir = os.path.join(exp_dir, 'learning')  # stores the learning progress
    res_dir = os.path.join(exp_dir, 'results')   # stores the experimenal results
    for d in (lrn_dir, res_dir):
        if not os.path.isdir(d):
            os.makedirs(d)
        elif len(os.listdir(d)) > 0:
            raise FileExistsError(f'Directory {d} is not empty!')
        else:
            pass
    return exp_dir, lrn_dir, res_dir

## Constants

In [13]:
# PRNG seed
SEED = 123

# model size (MPS_SIZE * LOCAL_DIM * BOND_DIM**2 ~ TRAIN_SIZE)
MPS_SIZE = 4
LOCAL_DIM = 4
BOND_DIM = 8

# data sample
TRAIN_SIZE = 1000
TEST_SIZE  = 5000

# training params
LEARNING_RATE = 5e-4

# max num of epochs
NUM_EPOCHS = 5000

# batch size
BATCH_SIZE = 50

# Approx rank
APPROX_RANK = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 25, 30]

# Noise model
PERCENT_NOISE = [0.1, 0.25, 0.5] # noise level in percentages to the data std

# Save/Print after that many epochs
SAVE_AFTER_EPOCHS = 100

# Weight initialisation
WEIGHT_INIT = 'he-uniform'

In [14]:
def save_settings(settings_file):
    with open(settings_file, 'w') as f:
        txt = f"""
SEED = {SEED}

# model size (MPS_SIZE * LOCAL_DIM + BOND_DIM = TRAIN_SIZE)
MPS_SIZE = {MPS_SIZE}
LOCAL_DIM = {LOCAL_DIM}
BOND_DIM = {BOND_DIM}

# data sample
TRAIN_SIZE = {TRAIN_SIZE}
TEST_SIZE = {TEST_SIZE}

# training params
LEARNING_RATE = {LEARNING_RATE}

# max num of epochs
NUM_EPOCHS = {NUM_EPOCHS}

# batch size
BATCH_SIZE = {BATCH_SIZE}

# Approx rank
APPROX_RANK = {APPROX_RANK}

# Noise model
PERCENT_NOISE = {PERCENT_NOISE}

# Save/Print after that many epochs
SAVE_AFTER_EPOCHS = {SAVE_AFTER_EPOCHS}

# Weight initialisation
WEIGHT_INIT = {WEIGHT_INIT}
"""
        f.write(txt)


In [15]:
WEIGHT_INITIALIZER = {
    'he-uniform': jax.nn.initializers.he_uniform,
    'he-normal': jax.nn.initializers.he_normal,
    'lecun-uniform': jax.nn.initializers.lecun_uniform,
    'lecun-normal': jax.nn.initializers.lecun_normal,
    'glorot-uniform': jax.nn.initializers.glorot_uniform,
    'glorot-normal': jax.nn.initializers.glorot_normal,
    'orthogonal': jax.nn.initializers.orthogonal
}

## Data Prep

In [16]:
key = jax.random.PRNGKey(SEED)

# Spliting the key
key_params, key_data, key_noise, key_run = jax.random.split(key, num=4)

# target MPS model
true_params = random_mps(key_params, size=MPS_SIZE, local_dim=LOCAL_DIM, bond_dim=BOND_DIM)

# generate samples
data = random_samples(key_data, sample_size=TRAIN_SIZE+TEST_SIZE, num_factors=MPS_SIZE, local_dim=LOCAL_DIM)

# train/test split
train_data, test_data = data[:TRAIN_SIZE], data[TRAIN_SIZE:]

# test targets
test_targets = dot_samples(true_params, test_data)

## Loss function

In [17]:
def loss(params: List[DeviceArray], data: Tuple[DeviceArray, DeviceArray]) -> jnp.double:
    inputs, targets = data
    outputs = dot_samples(params, inputs)
    err = jnp.subtract(targets, outputs)
    # return 0.5 * jnp.mean(jnp.log(jnp.power(err, 2) + 1))
    return jnp.sqrt(0.5 * jnp.mean(jnp.power(err, 2)))

## Execution

In [18]:
@jit
def update(i, opt_state, batch):
    params = get_params(opt_state)
    return opt_update(i, grad(loss)(params, batch), opt_state)

In [19]:
def gauss_noise(key: PRNGKeyArray, sample_size: int, scale=1.0) -> DeviceArray:
    return scale * jax.random.normal(key, shape=(sample_size,))

In [20]:
root_dir = './experiment'

# determining the step size for SGD
num_complete_batches, leftover = divmod(TRAIN_SIZE, BATCH_SIZE)
num_batches = num_complete_batches + bool(leftover)

In [21]:
def data_iterator():
    while True:
        perm = jax.random.permutation(key_run, TRAIN_SIZE)
        for i in range(num_batches):
            batch_idx = perm[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
            yield train_data[batch_idx], train_targets[batch_idx]

In [22]:
def init_like(
    key: PRNGKeyArray, 
    initializer: Callable, 
    mps: List[DeviceArray],
    dtype=jnp.double
    ) -> List[DeviceArray]:
    """MPS paramters initialization"""

    shapes = jax.tree_map(jnp.shape, mps) 
    keys = jax.random.split(key, num=len(shapes))

    return [initializer(*params) for params in zip(keys, shapes, [dtype]*len(mps))]   

In [23]:
# weight initializer
initializer = WEIGHT_INITIALIZER[WEIGHT_INIT]()

In [24]:
for perc_noise in PERCENT_NOISE:
    
    print(f'Noise level: {perc_noise}')
    print('='*100)
    
    # noise model
    noise = gauss_noise(key_noise, sample_size=TRAIN_SIZE, scale=perc_noise * jnp.std(jnp.asarray(train_data)))

    # generate outputs by contracting MPS with data
    train_targets = dot_samples(true_params, train_data) + noise
    
    # making exp directory
    exp_dir, lrn_dir, res_dir = make_dirs(root_dir, perc_noise)
    
    # storing the settings into a file
    settings_file = os.path.join(exp_dir, 'settings.txt')
    save_settings(settings_file)

    # storing results
    results = idict()

    ref_loss_tr = jnp.inf
    ref_loss_te = jnp.inf

    for approx_rank in APPROX_RANK:
    
        # timer
        tic = time.time()
        
        print(f'Approximation rank: {approx_rank}')
        print('='*100)
        
        loss_tr = []
        loss_te = []
        
        # get access to the data batches stream
        batches = data_iterator()
        
        # params for the optimization
        opt_init, opt_update, get_params = optimizers.sgd(step_size=LEARNING_RATE)
        
        # initialize approximating MPS parameters
        # first, a random MPS of the desired size is drawn
        # second, the MPS is initialised use a specific strategy (initialiser)
        params = random_mps(key_run, size=MPS_SIZE, local_dim=LOCAL_DIM, bond_dim=approx_rank)
        params = init_like(key_run, initializer, params, jnp.double)
        opt_state = opt_init(params)

        # iteration counter
        itercounter = itertools.count()
            
        # Main loop
        for epoch in range(NUM_EPOCHS):

            # update parameters
            for _ in range(num_batches):
                opt_state = update(next(itercounter), opt_state, next(batches))
            
            # get new params
            params = get_params(opt_state)
            
            # Generalization risk
            l_tr = loss(params, (train_data, train_targets))
            l_te = loss(params, (test_data, test_targets))
                    
            # storing errors for statistics (saving memory)
            loss_tr.append(l_tr)
            loss_te.append(l_te)
            
            # printing epochs
            if epoch % SAVE_AFTER_EPOCHS == 0:
                
                print(f'Epoch: {epoch:<15} \t|\t Train loss: {l_tr:<10.3f} \t|\t Test loss: {l_te:<10.3f}')
            
                # storing parameters during training
                file_path = os.path.join(lrn_dir, f'./approx_rank_{approx_rank}/epoch_{epoch}.pkl')
                save_pkl(file_path, params)
            
            # update the reference
            ref_loss_tr = l_tr
            ref_loss_te = l_te
            
        # storing train/test loss
        results["train"][approx_rank] = loss_tr
        results["test"][approx_rank] = loss_te

        print('-'*100)
        print(f'Time for rank {approx_rank}: {(time.time() - tic):0.2f} sec')
        print(f'Train loss: {ref_loss_tr:0.2f}')
        print(f'Test loss: {ref_loss_te:0.2f}')
        print('='*100)
        
        file_path = os.path.join(res_dir, f'approx_rank_{approx_rank}/loss.pkl')
        save_pkl(file_path, idict2dict(results))

Noise level: 0.1
Approximation rank: 2
Epoch: 0               	|	 Train loss: 30.233     	|	 Test loss: 29.851    
Epoch: 100             	|	 Train loss: 29.244     	|	 Test loss: 29.748    
Epoch: 200             	|	 Train loss: 27.889     	|	 Test loss: 30.539    
Epoch: 300             	|	 Train loss: 26.512     	|	 Test loss: 30.305    
Epoch: 400             	|	 Train loss: 25.305     	|	 Test loss: 29.518    
Epoch: 500             	|	 Train loss: 24.167     	|	 Test loss: 28.751    
Epoch: 600             	|	 Train loss: 22.767     	|	 Test loss: 27.906    
Epoch: 700             	|	 Train loss: 22.131     	|	 Test loss: 27.488    
Epoch: 800             	|	 Train loss: 21.794     	|	 Test loss: 27.364    
Epoch: 900             	|	 Train loss: 21.559     	|	 Test loss: 27.398    
Epoch: 1000            	|	 Train loss: 21.377     	|	 Test loss: 27.486    
Epoch: 1100            	|	 Train loss: 21.213     	|	 Test loss: 27.544    
Epoch: 1200            	|	 Train loss: 21.069    